In [2]:
from enum import Enum

class EventType(Enum):
    IMPORTANT_EVENT = 1
    LESS_IMPORTANT_EVENT = 2

class Event:
    def __init__(self, event_type):
        self.type = event_type

class EventObserver:
    def __init__(self):
        pass

    def notify(self, event, timeout_seconds):
        pass

    def notify_registration(self, new_observer):
        pass


class EventDispatcher:
    def __init__(self):
        self.observer_list = []

    def register_observer(self, event_type, observer):
        for existing_observer in self.observer_list:
            existing_observer[1].notify_registration(observer)

        self.observer_list.append((event_type, observer))

    def dispatch_event(self, event, timeout_seconds):
        for event_type, observer in self.observer_list:
            if event_type == event.type:
                observer.notify(event, timeout_seconds)

In [4]:
import unittest
from unittest import mock
from unittest.mock import Mock

class ObserverMock(EventObserver):
    def __init__(self):
        self.mock = Mock()

    def notify(self, event, time_out_in_seconds):
        self.mock.notify(event, time_out_in_seconds)

    def notify_registration(self, new_observer):
        self.mock.notify_registration(new_observer)


class TestEventDispatcher(unittest.TestCase):
    def setUp(self):
        self.event = Event(EventType.IMPORTANT_EVENT)
        self.dispatcher = EventDispatcher()
        self.observer = ObserverMock()
        self.observer2 = ObserverMock()

    def tearDown(self):
        pass

    def test_event_without_registrations_results_into_no_calls(self):
        self.dispatcher.dispatch_event(self.event, 10)

    def test_event_with_registration_for_event_results_into_callback(self):
        with mock.patch.object(self.observer.mock, 'notify') as mock_notify:
            self.event.type = EventType.IMPORTANT_EVENT

            self.dispatcher.register_observer(EventType.IMPORTANT_EVENT, self.observer)
            self.dispatcher.dispatch_event(self.event, 10)

            mock_notify.assert_called_once_with(self.event, 10)


    def test_different_event_with_registration_does_not_result_into_callback(self):
        self.event.type = EventType.LESS_IMPORTANT_EVENT
        self.dispatcher.register_observer(EventType.IMPORTANT_EVENT, self.observer)
        self.dispatcher.dispatch_event(self.event, 10)

    def test_register_two_observers_results_into_two_calls_and_registration_notification(self):
        with mock.patch.object(self.observer.mock, 'notify') as mock_notify, \
             mock.patch.object(self.observer2.mock, 'notify') as mock_notify2, \
             mock.patch.object(self.observer.mock, 'notify_registration') as mock_registration:

            self.event.type = EventType.IMPORTANT_EVENT
            self.dispatcher.register_observer(EventType.IMPORTANT_EVENT, self.observer)
            self.dispatcher.register_observer(EventType.IMPORTANT_EVENT, self.observer2)
            self.dispatcher.dispatch_event(self.event, 10)

            mock_notify.assert_called_once_with(self.event, 10)
            mock_notify2.assert_called_once_with(self.event, 10)
            mock_registration.assert_called_once_with(self.observer2)


unittest.main(argv=[''], verbosity=2, exit=False)

test_different_event_with_registration_does_not_result_into_callback (__main__.TestEventDispatcher) ... ok
test_event_with_registration_for_event_results_into_callback (__main__.TestEventDispatcher) ... ok
test_event_without_registrations_results_into_no_calls (__main__.TestEventDispatcher) ... ok
test_register_two_observers_results_into_two_calls_and_registration_notification (__main__.TestEventDispatcher) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.034s

OK
